# Sentence Embeddings using Siamese BERT-Networks

Based on the code from: 
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/training_nli_bert.py
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_clustering.py
- FAISs: https://github.com/bhavsarpratik/transformers/blob/master/5.%20semantic-search-USE.ipynb

This is a simple test application for sentence embeddings: semantic search in norwegian

We have a corpus with various sentences. Then, for a given query sentence, we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.


In [1]:
import numpy as np

# FAISS search

In [2]:
import faiss

class FAISS:
    def __init__(self, dimensions:int):
        self.dimensions = dimensions
        self.index = faiss.IndexFlatL2(dimensions)
        self.vectors = {}
        self.counter = 0
    
    def add(self, text:str, v:list):
        self.index.add(v)
        self.vectors[self.counter] = (text, v)
        self.counter += 1
        
    def search(self, v:list, k:int=10):
        distance, item_index = self.index.search(v, k)
        for dist, i in zip(distance[0], item_index[0]):
            if i==-1:
                break
            else:
                print(f'{self.vectors[i][0]}, %.2f'%dist)

In [3]:
from sentence_transformers import SentenceTransformer, models

class DistilBERT():
    """DistilBERT models
    device: cpu,cuda"""
    def __init__(self, model_name='distilbert-base-multilingual-cased', use_mean_pooling=True, use_cls_pooling=False, use_max_pooling=False, device='cpu'):
        model = models.DistilBERT(model_name)
        pooling =  models.Pooling(model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=use_mean_pooling,
                               pooling_mode_cls_token=use_cls_pooling,
                               pooling_mode_max_tokens=use_max_pooling)
        self.embeddings = SentenceTransformer(modules=[model, pooling], device=device) 
        
    def encode(self, text:list):
        result = self.embeddings.encode(text)
        return np.asarray(result, dtype=np.float32)

In [4]:
from tqdm import tqdm

class SemanticSearch():
    def __init__(self, encoder, dimension):
        self.encoder = encoder
        self.index = FAISS(dimension)
            
    def ingest(self, text:list):
        """text: a list of strings"""
        for t in tqdm(text):
            emb = self.encoder.encode([t])
            self.index.add(t, emb)
    
    def search(self, query, top:int=10):
        emb = self.encoder.encode([query])
        return self.index.search(emb, top)

# Test Vector Search

In [5]:
encoder = DistilBERT()

In [6]:
dimension = encoder.encode(['hello']).size
print(dimension)

768


In [7]:
index = FAISS(dimension)

# index word
t1 = 'hello'
v1 = encoder.encode([t1])
index.add(t1, v1)

# index word
t1 = 'bye'
v1 = encoder.encode([t1])
index.add(t1, v1)

# search similar word
t1 = 'hi'
v1 = encoder.encode([t1])
print('word,  distance')
index.search(v1)

word,  distance
hello, 39.76
bye, 61.25


# Test with real data 

In [8]:
corpus = ['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.',
          '58 personer var inntil tirsdag testet for coronaviruset i Norge.',
          'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .',
          'Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.',
          'Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.',
          'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.',
          'De vordende foreldrene er lettet over å få tilbringe natten under et tak.',
          'Kathy (31) er gravid og hjemløs.',
          'Nå har hun bodd på gaten i åtte måneder.'
          ]

In [9]:
encoder = DistilBERT()
dimension = encoder.encode(['hello']).size
ss = SemanticSearch(encoder,dimension)

In [10]:
ss.ingest(corpus)

100%|██████████| 9/9 [00:00<00:00, 36.14it/s]


In [23]:
ss.search('Tyrkia får norsk kritikk for Syria-aksjoner.')

Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen., 12.00
NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger., 19.18
Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag., 21.32
På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken ., 34.66
Legevakten i Trondheim ble onsdag ettermiddag delvis stengt., 38.61
58 personer var inntil tirsdag testet for coronaviruset i Norge., 39.94
De vordende foreldrene er lettet over å få tilbringe natten under et tak., 43.24
Kathy (31) er gravid og hjemløs., 43.72
Nå har hun bodd på gaten i åtte måneder., 50.74


# Test with News

## https://www.aftenposten.no/verden/i/b5b4Xg/libyas-fn-stoettede-regjering-trekker-seg-fra-fredssamtaler

In [12]:
article = 'Libyas FN-støttede regjering trekker seg fra fredssamtaler. Den FN-støttede regjeringen i Libya trekker seg inntil videre fra fredssamtalene i Genève etter at havnen i Tripoli tirsdag ble utsatt for et angrep. I dag var det på nytt brudd på våpenhvilen. Militsen angrep havnen i Tripoli, som er livsnerve for mange av Libyas byer, heter det i en kunngjøring fra regjeringen, som ledes av statsminister Fayez al-Serraj. Vi kunngjør at vi innstiller vår deltakelse i de militære samtalene som holdes i Genève inntil det blir inntatt en streng holdning mot angriperne og deres brudd, heter det videre. Innbyggere i Tripoli fortalte tirsdag om høye eksplosjoner og røyk fra havneområdet, men omfanget av skadene er ikke kjent. Oljerike Libya har vært preget av kaos og anarki siden opprørere med luftstøtte fra Norge og andre Nato-land i 2011 styrtet Muammar Gaddafis regime. Siden har en rekke militsgrupper kjempet om makten, noe som har bidratt til å gjøre Libya til et senter for menneskehandel og et brohode for flyktninger og migranter som forsøker å ta seg sjøveien til Europa. I april i fjor ble situasjonen ytterligere forverret da opprørsgeneralen Khalifa Haftar og hans milits innledet en offensiv mot Tripoli. En skjør våpenhvile er formelt på plass, men denne brytes ifølge FN regelmessig.'
lines = article.split('. ')
print(lines)

['Libyas FN-støttede regjering trekker seg fra fredssamtaler', 'Den FN-støttede regjeringen i Libya trekker seg inntil videre fra fredssamtalene i Genève etter at havnen i Tripoli tirsdag ble utsatt for et angrep', 'I dag var det på nytt brudd på våpenhvilen', 'Militsen angrep havnen i Tripoli, som er livsnerve for mange av Libyas byer, heter det i en kunngjøring fra regjeringen, som ledes av statsminister Fayez al-Serraj', 'Vi kunngjør at vi innstiller vår deltakelse i de militære samtalene som holdes i Genève inntil det blir inntatt en streng holdning mot angriperne og deres brudd, heter det videre', 'Innbyggere i Tripoli fortalte tirsdag om høye eksplosjoner og røyk fra havneområdet, men omfanget av skadene er ikke kjent', 'Oljerike Libya har vært preget av kaos og anarki siden opprørere med luftstøtte fra Norge og andre Nato-land i 2011 styrtet Muammar Gaddafis regime', 'Siden har en rekke militsgrupper kjempet om makten, noe som har bidratt til å gjøre Libya til et senter for menn

In [13]:
encoder_1 = DistilBERT()
dimension_1 = encoder_1.encode(['hello']).size
ss_1 = SemanticSearch(encoder_1,dimension_1)

In [14]:
ss_1.ingest(lines)

100%|██████████| 10/10 [00:00<00:00, 34.76it/s]


In [24]:
ss_1.search('brudd i våpenhvile', top=5)

I dag var det på nytt brudd på våpenhvilen, 20.77
En skjør våpenhvile er formelt på plass, men denne brytes ifølge FN regelmessig., 27.22
I april i fjor ble situasjonen ytterligere forverret da opprørsgeneralen Khalifa Haftar og hans milits innledet en offensiv mot Tripoli, 36.01
Vi kunngjør at vi innstiller vår deltakelse i de militære samtalene som holdes i Genève inntil det blir inntatt en streng holdning mot angriperne og deres brudd, heter det videre, 37.60
Den FN-støttede regjeringen i Libya trekker seg inntil videre fra fredssamtalene i Genève etter at havnen i Tripoli tirsdag ble utsatt for et angrep, 40.41


In [25]:
ss_1.search('absolutt ikke brudd i våpenhvile', top=5)

En skjør våpenhvile er formelt på plass, men denne brytes ifølge FN regelmessig., 18.19
I dag var det på nytt brudd på våpenhvilen, 18.59
Vi kunngjør at vi innstiller vår deltakelse i de militære samtalene som holdes i Genève inntil det blir inntatt en streng holdning mot angriperne og deres brudd, heter det videre, 25.17
I april i fjor ble situasjonen ytterligere forverret da opprørsgeneralen Khalifa Haftar og hans milits innledet en offensiv mot Tripoli, 27.59
Innbyggere i Tripoli fortalte tirsdag om høye eksplosjoner og røyk fra havneområdet, men omfanget av skadene er ikke kjent, 28.51


In [17]:
ss_1.search('Lybia i kaos og anarki', top=5)

Oljerike Libya har vært preget av kaos og anarki siden opprørere med luftstøtte fra Norge og andre Nato-land i 2011 styrtet Muammar Gaddafis regime, 32.77
I april i fjor ble situasjonen ytterligere forverret da opprørsgeneralen Khalifa Haftar og hans milits innledet en offensiv mot Tripoli, 34.25
Libyas FN-støttede regjering trekker seg fra fredssamtaler, 36.49
Innbyggere i Tripoli fortalte tirsdag om høye eksplosjoner og røyk fra havneområdet, men omfanget av skadene er ikke kjent, 37.73
Militsen angrep havnen i Tripoli, som er livsnerve for mange av Libyas byer, heter det i en kunngjøring fra regjeringen, som ledes av statsminister Fayez al-Serraj, 38.17


## Wikipedia https://no.wikipedia.org/wiki/Utlendingsdirektoratet

In [18]:
udi_article = 'Utlendingsdirektoratet (UDI) er et norsk statlig direktorat opprettet i 1988 og underlagt Justis- og beredskapsdepartementet da Statens utlendingskontor og Statens flyktningssekretariat ble slått sammen. Direktør er Frode Forfang. Direktoratet har ansvar for behandling av søknader om visum, oppholdstillatelse, statsborgerskap, reisedokumenter for utlendinger og beskyttelse (asyl) og for driften av asylmottak. UDI behandler også søknader om såkalt assistert retur[5], en ordning som gir økonomisk støtte til asylsøkere med avslag på søknaden sin som må returnere til hjemlandet og tilbakevending, en ordning som gir økonomisk støtte til flyktninger som ønsker å returnere til hjemlandet sitt. Direktoratet fatter dessuten også vedtak om utvisning. Fram til 1. januar 2006 hadde UDI også ansvar for integreringen av flyktninger i norske kommuner, men denne delen av virksomheten ble skilt ut som et nytt direktorat, Integrerings- og mangfoldsdirektoratet. UDI er underlagt Justis- og beredskapsdepartementet, som har ansvaret for flyktning- og innvandringspolitikken. Ansvaret for integrering, statsborgerloven og Integrerings- og mangfoldsdirektoratet (IMDi) ligger hos Kunnskapsdepartementet. Departementene har mulighet til å bestemme hvordan UDI skal tolke regelverket. Arbeids- og sosialdepartementet har ansvar for arbeidsinnvandring. Utlendingsloven, som de fleste av UDIs vedtak fattes etter, er vedtatt av Stortinget. Søkere som får avslag hos UDI, kan anke saken sin videre til Utlendingsnemnda (UNE) som er et frittstående, domstollignende forvaltningsorgan. UDI har ikke egne kontorer som tar imot søknader i Norge eller i utlandet. Politidistriktene tar imot og forbereder søknader om oppholdstillatelse, reisedokumenter, permanent opphold og statsborgerskap. I Oslo, Kirkenes, Stavanger, Bergen og Trondheim er det egne servicesentre for utenlandske arbeidstakere som tar imot søknader. I enkelte sakstyper kan politidistriktene også innvilge tillatelser når det ikke er tvil om at vilkårene er oppfylt. De har også ansvaret for registreringsordningen for EØS-borgere. Utenriksstasjonene (ambassader og konsulater) tar imot flere typer søknader og behandler de aller fleste søknadene om besøksvisum (maksimum 90 dager). Utenriksstasjonene hjelper også UDI med å hente inn og kontrollere opplysninger og dokumenter i oppholds- og asylsaker. Utenriksstasjonene ligger under Utenriksdepartementet. I saksbehandlingen benytter UDI blant annet landinformasjon fra Landinfo, som publiserer rapporter og temanotater om en rekke land. UDI er organisert i tre fagavdelinger, to støtteavdelinger samt HR-enheten, Administrative fellestjenester, Kommunikasjonstaben, Internrevisjonen og Direktørens stab. De tre fagavdelingene er Asylavdelingen (ASA), Oppholdsavdelingen (OPA) og Region- og mottaksavdelingen (RMA). De to støtteavdelingene er Avdeling for utvikling og analyse (AUA) og Avdeling for elektronisk forvaltning (AEF). Under RMA ligger direktoratets fem regionkontorer, som er lagt til Narvik, Trondheim, Gjøvik, Bergen og Kristiansand. Direktoratet har 1 037 ansatte (2013) og hovedkontor i Hausmanns gate nordøst i Oslo sentrum. Hovedkontoret flytter til Helsfyr på nyåret 2021. Fra 1917 til 1987 var det kjent som Sentralpasskontoret og Statens utlendingskontor. Før 1980 kom det bare 50 flyktninger til Norge i året. 1983 økte antallet til 150. Året etter kom det 300, deretter økte det til 900 i 1985, 2000 i 1986 og 8700 i 1987. Derfor ble UDI ble opprettet 2. januar 1988 under det daværende Kommunal- og arbeidsdepartementet for å ta hånd om innvandrings-, integrerings- og flyktningarbeid. Direktoratet fikk overført oppgavene til Statens utlendingskontor og Statens flyktningsekretariat samt en del av oppgavene fra innvandringsseksjonen i departementet. 1. juli 2000 overtok direktoratet oppgaven med å intervjue asylsøkere, en oppgave som tidligere var tillagt politiet. I januar 2006 ble direktoratets integreringsarbeid overført til et eget direktorat, Integrerings- og mangfoldsdirektoratet (IMDi). UDIs første direktør var Arild Kjerschow, som satt i stillingen fra 1987–1996. Da overtok Petter J. Drefvelin. Direktoratet ble fra høsten 2001 og frem til årsskiftet 2006-2007 ledet av Trygve G. Nordby. Manuela Ramin-Osmundsen, assisterende direktør fra 2002, ble tilsatt som direktør i mars 2006. Hun ledet direktoratet frem til 22. mai 2006, da Ida Børresen overtok som fungerende direktør. Børresen ble formelt tilsatt 10. november 2006. Børresen avgikk som direktør 30. mars 2012. Assisterende direktør Frode Forfang ble 22. juni 2012 utnevnt som ny direktør i UDI[1] og fikk nytt seksårsåremål i 2018. '
udi_lines = udi_article.split('. ')
print(udi_lines)

['Utlendingsdirektoratet (UDI) er et norsk statlig direktorat opprettet i 1988 og underlagt Justis- og beredskapsdepartementet da Statens utlendingskontor og Statens flyktningssekretariat ble slått sammen', 'Direktør er Frode Forfang', 'Direktoratet har ansvar for behandling av søknader om visum, oppholdstillatelse, statsborgerskap, reisedokumenter for utlendinger og beskyttelse (asyl) og for driften av asylmottak', 'UDI behandler også søknader om såkalt assistert retur[5], en ordning som gir økonomisk støtte til asylsøkere med avslag på søknaden sin som må returnere til hjemlandet og tilbakevending, en ordning som gir økonomisk støtte til flyktninger som ønsker å returnere til hjemlandet sitt', 'Direktoratet fatter dessuten også vedtak om utvisning', 'Fram til 1', 'januar 2006 hadde UDI også ansvar for integreringen av flyktninger i norske kommuner, men denne delen av virksomheten ble skilt ut som et nytt direktorat, Integrerings- og mangfoldsdirektoratet', 'UDI er underlagt Justis- o

In [19]:
ss_udi = SemanticSearch(encoder_1,dimension_1)

In [20]:
ss_udi.ingest(udi_lines)

100%|██████████| 53/53 [00:01<00:00, 48.43it/s]


In [26]:
ss_udi.search('hvem var UDIs første direktør?', top=5)

UDIs første direktør var Arild Kjerschow, som satt i stillingen fra 1987–1996, 36.65
Direktør er Frode Forfang, 37.85
I enkelte sakstyper kan politidistriktene også innvilge tillatelser når det ikke er tvil om at vilkårene er oppfylt, 40.43
Assisterende direktør Frode Forfang ble 22, 43.03
Børresen avgikk som direktør 30, 45.39


In [27]:
ss_udi.search('hvem er UDIs direktør?', top=5)

Direktør er Frode Forfang, 38.21
I enkelte sakstyper kan politidistriktene også innvilge tillatelser når det ikke er tvil om at vilkårene er oppfylt, 39.50
UDIs første direktør var Arild Kjerschow, som satt i stillingen fra 1987–1996, 42.24
Departementene har mulighet til å bestemme hvordan UDI skal tolke regelverket, 46.10
Assisterende direktør Frode Forfang ble 22, 47.41
